In [1]:
import pandas as pd
import numpy as np

# Load Maryland data

## Define zip codes

In [2]:
md_zip_ranges = [
    (20331,20331),
    (20335,20797),
    (20812,21930)
   
]

md_zips = ['%05d' % zipc for zip_range in md_zip_ranges
                     for zipc in range(zip_range[0],zip_range[1]+1)]

cases = pd.DataFrame(
    [np.NaN for _ in range(len(md_zips))],
    index=md_zips,
    columns=['cases']
)

md_zips = set(md_zips)

## MD County COVID-19 Cases By Zip Code

In [3]:
import requests

In [4]:
zips_json = requests.get('https://services.arcgis.com/njFNhDsUCentVYJW/arcgis/rest/services/ZIPCodes_MD_1/FeatureServer/0/query?f=json&where=ProtectedCount%3E%3D8&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=ZIPCODE1%20asc&resultOffset=0&resultRecordCount=300&cacheHint=true')


In [5]:
current_cases = pd.DataFrame.from_dict (dict((x['attributes']['ZIPCODE1'],x['attributes']['ProtectedCount'])
                        for x in zips_json.json()['features'] ), orient='index', columns=['cases']) 

In [6]:
cases.update(current_cases)
cases = cases.dropna()

## Maryland population data

In [7]:
pop = pd.read_csv('population_by_zip_2010.csv',dtype={'zipcode': str})
pop['ZIPCODE'] = pop['zipcode']
pop.set_index('ZIPCODE')

pop = pop[(pop['gender'] != 'male') & (pop['gender'] != 'female')]

In [8]:
pop_cases = cases.merge(pop,left_on=cases.index,right_on='ZIPCODE')
pop_cases.loc[pop_cases.ZIPCODE == '20770']

,cases,population,minimum_age,maximum_age,gender,zipcode,geo_id,ZIPCODE
46,52.0,25173,NaN,NaN,NaN,20770,8600000US20770,20770


In [9]:
pop_cases[pop_cases.ZIPCODE == '21122']

,cases,population,minimum_age,maximum_age,gender,zipcode,geo_id,ZIPCODE
116,61.0,60576,NaN,NaN,NaN,21122,8600000US21122,21122


# Load DC data

Add in DC zip codes

In [10]:
dc_zips = pd.read_csv("dc-zipcodes.csv").zipcode.tolist()


In [11]:
dc_zips = set([ str(x) for x in dc_zips])

## DC population data

In [12]:
df_dc_pop = pop[pop.apply(lambda x: x.ZIPCODE in dc_zips,axis=1 )] 

In [13]:
dc_pop = df_dc_pop.population.sum()
dc_pop

601239

## DC Covid Data

In [14]:
df_states = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")

In [15]:
df_dc = df_states[df_states.state == 'DC']
df_dc

,state,positive,positiveScore,negativeScore,negativeRegularScore,commercialScore,grade,score,negative,pending,...,death,hospitalized,total,totalTestResults,posNeg,fips,dateModified,dateChecked,notes,hash
7,DC,2058,1.0,1.0,1.0,1.0,A,4.0,9460,NaN,...,67.0,NaN,11518,11518,11518,11,2020-04-14T04:00:00Z,2020-04-14T18:48:00Z,"Please stop using the ""total"" field. Use ""tota...",214b5f2a1b8cbb9cee3522129fac911f5fe8f8d6


## Merge DC and Maryland cases

In [16]:
pop_cases = pop_cases.append({ 'cases': int(df_dc.positive),'population': dc_pop, 'zipcode': 'DC', 
                 'ZIPCODE': 'DC'}, ignore_index=True)

# Load shape data

## Maryland shape data

In [17]:
import shapefile
shape = shapefile.Reader("shapefiles/cb_2018_us_zcta510_500k.shp")

In [18]:
md_features = [] 


In [19]:
for rec in shape.iterShapeRecords():
    zipcode = rec.record['GEOID10']
    if zipcode in md_zips:
            md_features.append(rec.__geo_interface__)



## US States shape data

In [20]:
us_states_shapes = shapefile.Reader("shapefiles/cb_2018_us_state_5m.shp")

In [21]:
for rec in us_states_shapes.iterShapeRecords():
    geo_rec = rec.__geo_interface__ 
    # since we're merging on zipcode, copy state id to a dummy zipcode field
    geo_rec['properties']['ZCTA5CE10'] = rec.record['STUSPS']
    md_features.append(geo_rec)


## Create geo object

In [22]:
md_geos = { "type" : "FeatureCollection", "features" : md_features}

# Cases per 1000

In [23]:
pop_cases['density'] = pop_cases.cases / (pop_cases.population / 1_000)
# zips.set_index('ZIPCODE')
pop_cases[['ZIPCODE','density','cases','population']].sort_values('density')

,ZIPCODE,density,cases,population
163,21502,0.247748,11.0,44400
183,21811,0.443459,10.0,22550
165,21613,0.461627,8.0,17330
87,21009,0.470335,14.0,29766
89,21014,0.471607,17.0,36047
...,...,...,...,...
30,20722,3.502014,20.0,5711
175,21771,3.585563,106.0,29563
4,20613,3.625632,43.0,11860
19,20706,3.954306,153.0,38692


# Plot

In [24]:
import plotly.express as px

import plotly.graph_objects as go

In [25]:
fig = px.choropleth_mapbox(pop_cases,geojson=md_geos,
                           featureidkey='properties.ZCTA5CE10',
                           locations='ZIPCODE',
                           color='density',
                           mapbox_style="carto-positron", 
                           center={"lat": 39.0991, "lon" : -76.0255}, # -76.602565, 39.099115999999995
                           #opacity=0.5,
                           zoom=6
                          )
                        

In [26]:
fw = go.FigureWidget(fig)
fw

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'featureidkey': 'properties.ZCTA5CE10',
 …

With plotly, you have to manually enter the center coordinates.  This zip is close to the center; find it and print out it's geo shape.

In [40]:
for rec in md_features:
    if rec['properties']['ZCTA5CE10'] == '21146' :
        print(rec)
        break

{'type': 'Feature', 'properties': {'ZCTA5CE10': '21146', 'AFFGEOID10': '8600000US21146', 'GEOID10': '21146', 'ALAND10': 26503472, 'AWATER10': 7093956}, 'geometry': {'type': 'Polygon', 'coordinates': (((-76.615927, 39.090083), (-76.614615, 39.092098), (-76.611378, 39.090492999999995), (-76.60674, 39.092256), (-76.605198, 39.093202), (-76.605097, 39.099216999999996), (-76.602565, 39.099115999999995), (-76.603659, 39.101469), (-76.602203, 39.101735999999995), (-76.601756, 39.10306), (-76.597636, 39.100688), (-76.597336, 39.09699), (-76.593789, 39.095208), (-76.58690299999999, 39.094179), (-76.580635, 39.094798999999995), (-76.577691, 39.094364999999996), (-76.573016, 39.094961999999995), (-76.56939799999999, 39.097564999999996), (-76.570751, 39.099447999999995), (-76.567031, 39.101129), (-76.565129, 39.1008), (-76.565017, 39.10447), (-76.558677, 39.104437), (-76.555284, 39.102447999999995), (-76.552005, 39.103254), (-76.55071099999999, 39.101929), (-76.54749199999999, 39.101914), (-76.546

# State map

In [80]:
fig2 = px.choropleth_mapbox(df_states,geojson=md_geos,
                           featureidkey='properties.STUSPS',
                           locations='state',
                           color='positive',
                           mapbox_style="carto-positron", 
                           center={"lat": 39.0991, "lon" : -76.0255}, # -76.602565, 39.099115999999995
                           #opacity=0.5,
                           zoom=5
                          )
                        

In [81]:
go.FigureWidget(fig2)

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'featureidkey': 'properties.STUSPS',
    …

# Misc data

Here are misclaneous Web sites with covid related data that I found.

In [52]:
# https://services9.arcgis.com/6Hv9AANartyT7fJW/arcgis/rest/services/USCounties_cases_V1/FeatureServer/0/query?f=json&returnIdsOnly=false&returnGeometry=false&where=1%3D1&spatialRel=esriSpatialRelIntersects&outSR=102100&cacheHint=true
# The were clause aboue &where=1%3D1   => where 1=1
# https://covidtracking.com/api
# https://coronavirus.dc.gov/sites/default/files/dc/sites/coronavirus/page_content/attachments/DC-COVID-19-Data-forApril-12-2020.xlsx

df_dc = pd.read_excel("DC-COVID-19-Data-forApril-12-2020.xlsx")

In [66]:
df_dc.iloc[0:20,[0,1,38]]

,Unnamed: 0,Unnamed: 1,2020-04-12 00:00:00
0,NaN,Testing,NaN
1,Testing,People Tested Overall,10934.0
2,Testing,Total Positives,1955.0
3,Testing,Number of Deaths,52.0
4,Testing,People Recovered,507.0
5,NaN,NaN,NaN
6,Hospitals,ICU Beds Available,105.0
7,Hospitals,Total Ventilators,444.0
8,Hospitals,Ventilators in Use,218.0
9,Hospitals,Ventilators Available,226.0


In [63]:
df_dc.shape

(74, 39)